In [ ]:
# | default_exp _application.tester

In [ ]:
# | export

import asyncio
import collections
import inspect
from unittest.mock import AsyncMock, MagicMock
import json
from contextlib import asynccontextmanager
from itertools import groupby
from typing import *
from types import ModuleType

from pydantic import BaseModel

from fastkafka import KafkaEvent
from fastkafka._application.app import FastKafka, AwaitedMock, _get_kafka_brokers
from fastkafka._components.asyncapi import KafkaBroker, KafkaBrokers
from fastkafka._components.helpers import unwrap_list_type
from fastkafka._components.meta import delegates, export, patch
from fastkafka._components.producer_decorator import unwrap_from_kafka_event
from fastkafka._components.aiokafka_consumer_loop import ConsumeCallable
from fastkafka._testing.apache_kafka_broker import ApacheKafkaBroker
from fastkafka._testing.in_memory_broker import InMemoryBroker
from fastkafka._testing.local_redpanda_broker import LocalRedpandaBroker
from fastkafka._components.helpers import remove_suffix

In [ ]:
import pytest
from pydantic import Field

from fastkafka import EventMetadata, KafkaEvent
from fastkafka._components.logger import get_logger, suppress_timestamps

In [ ]:
suppress_timestamps()
logger = get_logger(__name__, level=20)
logger.info("ok")

[INFO] __main__: ok


In [ ]:
# | notest
# allows async calls in notebooks

import nest_asyncio

In [ ]:
# | notest

nest_asyncio.apply()

In [ ]:
class TestMsg(BaseModel):
    msg: str = Field(...)


app = FastKafka(kafka_brokers=dict(localhost=dict(url="localhost", port="9092")))


@app.consumes()
async def on_preprocessed_signals(msg: TestMsg):
    await to_predictions(TestMsg(msg="prediction"))


@app.produces()
async def to_predictions(prediction: TestMsg) -> TestMsg:
    print(f"Sending prediction: {prediction}")
    return prediction

In [ ]:
# | export


def _get_broker_spec(bootstrap_server: str) -> KafkaBroker:
    """
    Helper function to get the broker specification from the bootstrap server URL.

    Args:
        bootstrap_server: The bootstrap server URL in the format "<host>:<port>".

    Returns:
        A KafkaBroker object representing the broker specification.
    """
    url = bootstrap_server.split(":")[0]
    port = bootstrap_server.split(":")[1]
    return KafkaBroker(url=url, port=port, description="", protocol="")

## Fastkafka Tester class

In [ ]:
# | export


@export("fastkafka.testing")
class Tester(FastKafka):
    __test__ = False

    def __init__(
        self,
        app: Union[FastKafka, List[FastKafka]],
        *,
        use_in_memory_broker: bool = True,
    ):
        """Mirror-like object for testing a FastKafka application

        Can be used as context manager

        Args:
            app: The FastKafka application to be tested.
            use_in_memory_broker: Whether to use an in-memory broker for testing or not.
        """
        self.apps = app if isinstance(app, list) else [app]

        for app in self.apps:
            app.create_mocks()

        super().__init__()
        self.mirrors: Dict[Any, Any] = {}
        self._kafka_brokers = self.apps[0]._kafka_brokers
        self._kafka_config["bootstrap_servers_id"] = self.apps[0]._kafka_config[
            "bootstrap_servers_id"
        ]
        self._create_mirrors()
        self.use_in_memory_broker = use_in_memory_broker

    async def _start_tester(self) -> None:
        """Starts the Tester"""
        for app in self.apps:
            await app.__aenter__()
        self.create_mocks()
        self._arrange_mirrors()
        await super().__aenter__()
        await asyncio.sleep(3)

    async def _stop_tester(self) -> None:
        """Shuts down the Tester"""
        await super().__aexit__(None, None, None)
        for app in self.apps[::-1]:
            await app.__aexit__(None, None, None)

    def _create_mirrors(self) -> None:
        pass

    def _arrange_mirrors(self) -> None:
        pass
    
    def _set_arguments_and_return_old(self,
                      bootstrap_servers_id: Optional[str],
                      use_in_memory_broker: bool
                      ) -> Dict[Any, Any]:
        initial_arguments: Dict[Any, Any] = dict()
        initial_arguments["use_in_memory_broker"] = self.use_in_memory_broker
        self.use_in_memory_broker = use_in_memory_broker
        
        initial_arguments["bootstrap_servers_id"] = self._kafka_config["bootstrap_servers_id"]
        if bootstrap_servers_id is None:
            bootstrap_servers_id = self._kafka_config["bootstrap_servers_id"]
        else:
            self._kafka_config["bootstrap_servers_id"] = bootstrap_servers_id
        
        for app in self.apps:
            initial_arguments[app] = app._kafka_config["bootstrap_servers_id"]
            app._kafka_config["bootstrap_servers_id"] = bootstrap_servers_id
            
        return initial_arguments
    
    def _restore_initial_arguments(self,
                                  initial_arguments: Dict[Any, Any]
                                  ) -> None:
        self.use_in_memory_broker = initial_arguments["use_in_memory_broker"]
        self._kafka_config["bootstrap_servers_id"] = initial_arguments["bootstrap_servers_id"]
        
        for app in self.apps:
            app._kafka_config["bootstrap_servers_id"] = initial_arguments[app]

    @asynccontextmanager
    async def using_external_broker(self,
                                    bootstrap_servers_id: Optional[str] = None,
                     ) -> AsyncGenerator["Tester", None]:
        """Tester context manager for using external broker 

        Args:
            bootstrap_servers_id: The bootstrap server of aplications.
            
        Returns:
            self or None
        """
        initial_arguments = self._set_arguments_and_return_old(bootstrap_servers_id, use_in_memory_broker=False)
            
        async with self._create_ctx() as ctx:
            try:
                yield self
            finally:
                self._restore_initial_arguments(initial_arguments)
                
                
    @asynccontextmanager
    async def using_inmemory_broker(self, 
                      bootstrap_servers_id: Optional[str] = None,
                     ) -> AsyncGenerator["Tester", None]:
        """Tester context manager for using in-memory broker 

        Args:
            bootstrap_servers_id: The bootstrap server of aplications.
            
        Returns:
            self or None
        """
        initial_arguments = self._set_arguments_and_return_old(bootstrap_servers_id, use_in_memory_broker=True)
            
        async with self._create_ctx() as ctx:
            try:
                yield self
            finally:
                self._restore_initial_arguments(initial_arguments)
            
            
    @asynccontextmanager
    async def _create_ctx(self) -> AsyncGenerator["Tester", None]:
        if self.use_in_memory_broker == True:
            with InMemoryBroker(): # type: ignore
                await self._start_tester()
                try:
                    yield self
                finally:
                    await self._stop_tester()
        else:
            await self._start_tester()
            try:
                yield self
            finally:
                await self._stop_tester()

    async def __aenter__(self) -> "Tester":
        self._ctx = self._create_ctx()
        return await self._ctx.__aenter__()

    async def __aexit__(self, *args: Any) -> None:
        await self._ctx.__aexit__(*args)

In [ ]:
for _ in range(2):
    with pytest.raises(RuntimeError) as e:
        async with Tester(app) as tester:
            assert tester.is_started
            raise RuntimeError("ok")

    print(e)
    assert not tester.is_started

[INFO] fastkafka._testing.in_memory_broker: InMemoryBroker._patch_consumers_and_producers(): Patching consumers and producers!
[INFO] fastkafka._testing.in_memory_broker: InMemoryBroker starting
[INFO] fastkafka._application.app: _create_producer() : created producer using the config: '{'bootstrap_servers': 'localhost:9092'}'
[INFO] fastkafka._testing.in_memory_broker: AIOKafkaProducer patched start() called()
[INFO] fastkafka._components.aiokafka_consumer_loop: aiokafka_consumer_loop() starting...
[INFO] fastkafka._components.aiokafka_consumer_loop: aiokafka_consumer_loop(): Consumer created using the following parameters: {'auto_offset_reset': 'earliest', 'max_poll_records': 100, 'bootstrap_servers': 'localhost:9092'}
[INFO] fastkafka._testing.in_memory_broker: AIOKafkaConsumer patched start() called()
[INFO] fastkafka._components.aiokafka_consumer_loop: aiokafka_consumer_loop(): Consumer started.
[INFO] fastkafka._testing.in_memory_broker: AIOKafkaConsumer patched subscribe() called

In [ ]:
tester = Tester(app)


@tester.produces()
async def to_preprocessed_signals(msg: TestMsg) -> TestMsg:
    print(f"Producing msg {msg}")
    return msg


tester.to_preprocessed_signals = to_preprocessed_signals


@tester.consumes(auto_offset_reset="latest")
async def on_predictions(msg: TestMsg):
    pass


async with tester:
    await tester.to_preprocessed_signals(TestMsg(msg="signal"))
    await asyncio.sleep(5)
    tester.mocks.on_predictions.assert_called()

print("ok")

[INFO] fastkafka._testing.in_memory_broker: InMemoryBroker._patch_consumers_and_producers(): Patching consumers and producers!
[INFO] fastkafka._testing.in_memory_broker: InMemoryBroker starting
[INFO] fastkafka._application.app: _create_producer() : created producer using the config: '{'bootstrap_servers': 'localhost:9092'}'
[INFO] fastkafka._testing.in_memory_broker: AIOKafkaProducer patched start() called()
[INFO] fastkafka._application.app: _create_producer() : created producer using the config: '{'bootstrap_servers': 'localhost:9092'}'
[INFO] fastkafka._testing.in_memory_broker: AIOKafkaProducer patched start() called()
[INFO] fastkafka._components.aiokafka_consumer_loop: aiokafka_consumer_loop() starting...
[INFO] fastkafka._components.aiokafka_consumer_loop: aiokafka_consumer_loop(): Consumer created using the following parameters: {'auto_offset_reset': 'earliest', 'max_poll_records': 100, 'bootstrap_servers': 'localhost:9092'}
[INFO] fastkafka._testing.in_memory_broker: AIOKafk

## Test multiple brokers

In [ ]:
kafka_brokers_1 = dict(localhost=[dict(url="server_1", port="9092")])
kafka_brokers_2 = dict(localhost=dict(url="server_2", port="9092"))

app = FastKafka(kafka_brokers=kafka_brokers_1)


@app.consumes(topic="preprocessed_signals")
async def on_preprocessed_signals_1(msg: TestMsg):
    print(f"Default broker:  {msg=}")
    
@app.consumes(topic="preprocessed_signals", brokers=kafka_brokers_2)
async def on_preprocessed_signals_2(msg: TestMsg):
    print(f"Defined broker:  {msg=}")

tester = Tester(app)


@tester.produces(brokers=kafka_brokers_2)
async def to_preprocessed_signals(msg: TestMsg) -> TestMsg:
    print(f"Producing msg {msg}")
    return msg


tester.to_preprocessed_signals = to_preprocessed_signals

async with tester:
    await tester.to_preprocessed_signals(TestMsg(msg="signal"))
    await asyncio.sleep(5)
    await app.awaited_mocks.on_preprocessed_signals_2.assert_called(
        timeout=5
    )
    await app.awaited_mocks.on_preprocessed_signals_1.assert_not_called(
        timeout=5
    )

print("ok")

[INFO] fastkafka._testing.in_memory_broker: InMemoryBroker._patch_consumers_and_producers(): Patching consumers and producers!
[INFO] fastkafka._testing.in_memory_broker: InMemoryBroker starting
[INFO] fastkafka._application.app: _create_producer() : created producer using the config: '{'bootstrap_servers': 'server_2:9092'}'
[INFO] fastkafka._testing.in_memory_broker: AIOKafkaProducer patched start() called()
[INFO] fastkafka._components.aiokafka_consumer_loop: aiokafka_consumer_loop() starting...
[INFO] fastkafka._components.aiokafka_consumer_loop: aiokafka_consumer_loop(): Consumer created using the following parameters: {'auto_offset_reset': 'earliest', 'max_poll_records': 100, 'bootstrap_servers': ['server_1:9092']}
[INFO] fastkafka._testing.in_memory_broker: AIOKafkaConsumer patched start() called()
[INFO] fastkafka._components.aiokafka_consumer_loop: aiokafka_consumer_loop(): Consumer started.
[INFO] fastkafka._testing.in_memory_broker: AIOKafkaConsumer patched subscribe() called

In [ ]:
kafka_brokers_1 = dict(localhost=dict(url="server_1", port="9092"))
kafka_brokers_2 = dict(localhost=[dict(url="server_2", port="9092")])

app = FastKafka(kafka_brokers=kafka_brokers_1)


@app.consumes(topic="preprocessed_signals", brokers=kafka_brokers_2)
async def on_preprocessed_signals(msg: TestMsg):
    print(f"{msg=}")
    await to_predictions(TestMsg(msg="prediction"))


@app.produces()
async def to_predictions(prediction: TestMsg) -> TestMsg:
    print(f"Sending prediction: {prediction}")
    return prediction


tester = Tester(app)


@tester.produces(topic="preprocessed_signals", brokers=kafka_brokers_2)
async def to_preprocessed_signals(msg: TestMsg) -> TestMsg:
    print(f"Producing msg {msg}")
    return msg


@tester.consumes(auto_offset_reset="earliest", brokers=kafka_brokers_1)
async def on_predictions(msg: TestMsg):
    print(f"tester: {msg=}")


tester.to_preprocessed_signals = to_preprocessed_signals

async with tester:
    await tester.to_preprocessed_signals(TestMsg(msg="signal"))
    await asyncio.sleep(5)
    await app.awaited_mocks.on_preprocessed_signals.assert_called(timeout=5)
    await tester.awaited_mocks.on_predictions.assert_called(timeout=5)

print("ok")

[INFO] fastkafka._testing.in_memory_broker: InMemoryBroker._patch_consumers_and_producers(): Patching consumers and producers!
[INFO] fastkafka._testing.in_memory_broker: InMemoryBroker starting
[INFO] fastkafka._application.app: _create_producer() : created producer using the config: '{'bootstrap_servers': 'server_1:9092'}'
[INFO] fastkafka._testing.in_memory_broker: AIOKafkaProducer patched start() called()
[INFO] fastkafka._application.app: _create_producer() : created producer using the config: '{'bootstrap_servers': ['server_2:9092']}'
[INFO] fastkafka._testing.in_memory_broker: AIOKafkaProducer patched start() called()
[INFO] fastkafka._components.aiokafka_consumer_loop: aiokafka_consumer_loop() starting...
[INFO] fastkafka._components.aiokafka_consumer_loop: aiokafka_consumer_loop(): Consumer created using the following parameters: {'auto_offset_reset': 'earliest', 'max_poll_records': 100, 'bootstrap_servers': ['server_2:9092']}
[INFO] fastkafka._testing.in_memory_broker: AIOKaf

## Mirroring

In [ ]:
# | export


def mirror_producer(
    topic: str, producer_f: Callable[..., Any], brokers: str, app: FastKafka
) -> Callable[..., Any]:
    """
    Decorator to create a mirrored producer function.

    Args:
        topic: The topic to produce to.
        producer_f: The original producer function.
        brokers: The brokers configuration.
        app: The FastKafka application.

    Returns:
        The mirrored producer function.
    """
    msg_type = inspect.signature(producer_f).return_annotation

    msg_type_unwrapped = unwrap_list_type(unwrap_from_kafka_event(msg_type))

    async def skeleton_func(msg: BaseModel) -> None:
        pass

    mirror_func = skeleton_func
    sig = inspect.signature(skeleton_func)

    # adjust name, take into consideration the origin app and brokers
    # configuration so that we can differentiate those two
    mirror_func.__name__ = f"mirror_{id(app)}_on_{remove_suffix(topic).replace('.', '_')}_{abs(hash(brokers))}"

    # adjust arg and return val
    sig = sig.replace(
        parameters=[
            inspect.Parameter(
                name="msg",
                annotation=msg_type_unwrapped,
                kind=inspect.Parameter.POSITIONAL_OR_KEYWORD,
            )
        ]
    )

    mirror_func.__signature__ = sig  # type: ignore

    return mirror_func

In [ ]:
app = FastKafka(kafka_brokers=dict(localhost=dict(url="localhost", port=9092)))


@app.produces()
async def to_topic1() -> TestMsg:
    pass


@app.produces(topic="topic2")
async def some_log(in_var: int) -> TestMsg:
    pass


@app.produces(topic="topic2", brokers=dict(localhost=dict(url="localhost", port=9093)))
async def some_log_1(in_var: int) -> TestMsg:
    pass


@app.produces(topic="topic2", brokers=dict(localhost=dict(url="localhost", port=9093)))
async def some_log_2(in_var: int) -> TestMsg:
    pass


for topic, (producer_f, _, brokers, _) in app._producers_store.items():
    mirror = mirror_producer(
        topic,
        producer_f,
        brokers.model_dump_json() if brokers is not None else app._kafka_brokers.model_dump_json(),
        app,
    )
    assert "_".join(mirror.__name__.split("_")[2:-1]) == "on_" + remove_suffix(topic)
    assert (
        inspect.signature(mirror).parameters["msg"].annotation.__name__
        == inspect.Parameter(
            name="msg",
            annotation=TestMsg,
            kind=inspect.Parameter.POSITIONAL_OR_KEYWORD,
        ).annotation.__name__
    )

In [ ]:
app = FastKafka(kafka_brokers=dict(localhost=dict(url="localhost", port=9092)))

@app.produces()
async def to_topic1() -> TestMsg:
    pass


@app.produces(topic="topic2")
async def some_log(in_var: int) -> KafkaEvent[List[TestMsg]]:
    pass


for topic, (producer_f, _, brokers, _) in app._producers_store.items():
    mirror = mirror_producer(
        topic,
        producer_f,
        brokers.model_dump_json() if brokers is not None else app._kafka_brokers.model_dump_json(),
        app
    )
    assert "_".join(mirror.__name__.split("_")[2:-1]) == "on_" + remove_suffix(topic)
    assert inspect.signature(mirror).parameters["msg"].annotation.__name__ == "TestMsg"

In [ ]:
# | export


def mirror_consumer(
    topic: str, consumer_f: Callable[..., Any], brokers: str, app: FastKafka
) -> Callable[[BaseModel], Coroutine[Any, Any, BaseModel]]:
    """
    Decorator to create a mirrored consumer function.

    Args:
        topic: The topic to consume from.
        consumer_f: The original consumer function.
        brokers: The brokers configuration.
        app: The FastKafka application.

    Returns:
        The mirrored consumer function.
    """
    msg_type = inspect.signature(consumer_f).parameters["msg"]

    msg_type_unwrapped = unwrap_list_type(msg_type)

    async def skeleton_func(msg: BaseModel) -> BaseModel:
        return msg

    mirror_func = skeleton_func
    sig = inspect.signature(skeleton_func)

    # adjust name, take into consideration the origin app and brokers
    # configuration so that we can differentiate those two
    mirror_func.__name__ = f"mirror_{id(app)}_to_{remove_suffix(topic).replace('.', '_')}_{abs(hash(brokers))}"

    # adjust arg and return val
    sig = sig.replace(
        parameters=[msg_type], return_annotation=msg_type_unwrapped.annotation
    )

    mirror_func.__signature__ = sig  # type: ignore
    return mirror_func

In [ ]:
for topic, (consumer_f, _, _, brokers, _) in app._consumers_store.items():
    mirror = mirror_consumer(
        topic,
        consumer_f,
        brokers.model_dump_json() if brokers is not None else app._kafka_brokers.model_dump_json(),
        app
    )
    assert "_".join(mirror.__name__.split("_")[3:-1]) == "to_" + remove_suffix(topic)
    assert (
        inspect.signature(mirror).return_annotation.__name__ == TestMsg.__name__
    ), inspect.signature(mirror).return_annotation.__name__
    assert (
        inspect.signature(mirror).parameters["msg"].annotation.__name__
        == inspect.Parameter(
            name="msg",
            annotation=TestMsg,
            kind=inspect.Parameter.POSITIONAL_OR_KEYWORD,
        ).annotation.__name__
    )

In [ ]:
# | export


@patch
def _create_mirrors(self: Tester) -> None:
    """
    Creates mirror functions for producers and consumers.

    Iterates over the FastKafka application and its producers and consumers. For each consumer, it creates a mirror
    consumer function using the `mirror_consumer` decorator. For each producer, it creates a mirror producer function
    using the `mirror_producer` decorator. The mirror functions are stored in the `self.mirrors` dictionary and also
    set as attributes on the Tester instance.

    Returns:
        None
    """
    for app in self.apps:
        for topic, (consumer_f, _, _, brokers, _) in app._consumers_store.items():
            mirror_f = mirror_consumer(
                topic,
                consumer_f,
                brokers.model_dump_json() if brokers is not None else app._kafka_brokers.model_dump_json(),
                app,
            )
            mirror_f = self.produces(  # type: ignore
                topic=remove_suffix(topic),
                brokers=brokers,
            )(mirror_f)
            self.mirrors[consumer_f] = mirror_f
            setattr(self, mirror_f.__name__, mirror_f)
        for topic, (producer_f, _, brokers, _) in app._producers_store.items():
            mirror_f = mirror_producer(
                topic,
                producer_f,
                brokers.model_dump_json() if brokers is not None else app._kafka_brokers.model_dump_json(),
                app,
            )
            mirror_f = self.consumes(
                topic=remove_suffix(topic),
                brokers=brokers,
            )(
                mirror_f  # type: ignore
            )
            self.mirrors[producer_f] = mirror_f
            setattr(self, mirror_f.__name__, mirror_f)

In [ ]:
kafka_brokers_1 = dict(localhost=[dict(url="server_1", port=9092)])
kafka_brokers_2 = dict(localhost=dict(url="server_2", port=9092))
app = FastKafka(kafka_brokers=kafka_brokers_1)


@app.consumes(topic="preprocessed_signals", brokers=kafka_brokers_2)
async def on_preprocessed_signals(msg: TestMsg):
    print(f"{msg=}")
    await to_predictions(TestMsg(msg="prediction"))


@app.produces()
async def to_predictions(prediction: TestMsg) -> TestMsg:
    print(f"Sending prediction: {prediction}")
    return prediction


tester = Tester(app)

assert hasattr(
    tester,
    f"mirror_{id(app)}_to_preprocessed_signals_{abs(hash(_get_kafka_brokers(kafka_brokers_2).model_dump_json()))}",
)

assert hasattr(
    tester,
    f"mirror_{id(app)}_on_predictions_{abs(hash(_get_kafka_brokers(app._kafka_brokers).model_dump_json()))}",
)

In [ ]:
kafka_brokers_1 = dict(localhost=[dict(url="server_1", port=9092)])
kafka_brokers_2 = dict(localhost=dict(url="server_2", port=9092))

app = FastKafka(kafka_brokers=kafka_brokers_1)


@app.consumes(topic="preprocessed_signals", brokers=kafka_brokers_2)
async def on_preprocessed_signals(msg: TestMsg):
    print(f"{msg=}")
    await to_predictions(TestMsg(msg="prediction"))


@app.produces()
async def to_predictions(prediction: TestMsg) -> TestMsg:
    print(f"Sending prediction: {prediction}")
    return prediction


async with Tester(app) as tester:
    await getattr(
        tester,
        f"mirror_{id(app)}_to_preprocessed_signals_{abs(hash(_get_kafka_brokers(kafka_brokers_2).model_dump_json()))}",
    )(TestMsg(msg="signal"))
    await asyncio.sleep(5)
    await app.awaited_mocks.on_preprocessed_signals.assert_called(timeout=5)
    await getattr(
        tester.awaited_mocks,
        f"mirror_{id(app)}_on_predictions_{abs(hash(_get_kafka_brokers(kafka_brokers_1).model_dump_json()))}",
    ).assert_called(timeout=5)

print("ok")

[INFO] fastkafka._testing.in_memory_broker: InMemoryBroker._patch_consumers_and_producers(): Patching consumers and producers!
[INFO] fastkafka._testing.in_memory_broker: InMemoryBroker starting
[INFO] fastkafka._application.app: _create_producer() : created producer using the config: '{'bootstrap_servers': ['server_1:9092']}'
[INFO] fastkafka._testing.in_memory_broker: AIOKafkaProducer patched start() called()
[INFO] fastkafka._application.app: _create_producer() : created producer using the config: '{'bootstrap_servers': 'server_2:9092'}'
[INFO] fastkafka._testing.in_memory_broker: AIOKafkaProducer patched start() called()
[INFO] fastkafka._components.aiokafka_consumer_loop: aiokafka_consumer_loop() starting...
[INFO] fastkafka._components.aiokafka_consumer_loop: aiokafka_consumer_loop(): Consumer created using the following parameters: {'auto_offset_reset': 'earliest', 'max_poll_records': 100, 'bootstrap_servers': 'server_2:9092'}
[INFO] fastkafka._testing.in_memory_broker: AIOKafka

## Mirrors dict and syntax sugar

In [ ]:
# | export


class AmbiguousWarning:
    """
    Warning class used for ambiguous topics.

    Args:
        topic: The ambiguous topic.
        functions: List of function names associated with the ambiguous topic.
    """
    def __init__(self, topic: str, functions: List[str]):
        self.topic = topic
        self.functions = functions

    def __getattribute__(self, attr: str) -> Any:
        raise RuntimeError(
            f"Ambiguous topic: {super().__getattribute__('topic')}, for functions: {super().__getattribute__('functions')}\nUse Tester.mirrors[app.function] to resolve ambiguity"
        )

    def __call__(self, *args: Any, **kwargs: Any) -> Any:
        raise RuntimeError(
            f"Ambiguous topic: {self.topic}, for functions: {self.functions}\nUse Tester.mirrors[app.function] to resolve ambiguity"
        )

In [ ]:
with pytest.raises(Exception) as e:
    AmbiguousWarning(topic="some_topic", functions=["some_functions"])(TestMsg(msg="signal"))
    
assert e.value.args[0] == "Ambiguous topic: some_topic, for functions: ['some_functions']\nUse Tester.mirrors[app.function] to resolve ambiguity"

with pytest.raises(Exception) as e:
    AmbiguousWarning(topic="some_topic", functions=["some_brokers"]).assert_called(timeout=5)
    
assert e.value.args[0] == "Ambiguous topic: some_topic, for functions: ['some_brokers']\nUse Tester.mirrors[app.function] to resolve ambiguity"

In [ ]:
# | export


def set_sugar(
    *,
    tester: Tester,
    prefix: str,
    topic_brokers: Dict[str, Tuple[List[str], List[str]]],
    topic: str,
    brokers: str,
    origin_function_name: str,
    function: Callable[..., Union[Any, Awaitable[Any]]],
) -> None:
    """
    Sets the sugar function for a topic.

    Args:
        tester: The Tester instance.
        prefix: The prefix to use for the sugar function (e.g., "to_" or "on_").
        topic_brokers: Dictionary to store the brokers and functions associated with each topic.
        topic: The topic name.
        brokers: The brokers configuration.
        origin_function_name: The name of the original function.
        function: The mirror function to be set as the sugar function.

    Returns:
        None
    """
    brokers_for_topic, functions_for_topic = topic_brokers.get(topic, ([], []))
    if brokers not in brokers_for_topic:
        brokers_for_topic.append(brokers)
        functions_for_topic.append(origin_function_name)
        topic_brokers[topic] = (brokers_for_topic, functions_for_topic)
    if len(brokers_for_topic) == 1:
        setattr(tester, f"{prefix}{topic}", function)
    else:
        setattr(
            tester, f"{prefix}{topic}", AmbiguousWarning(topic, functions_for_topic)
        )

In [ ]:
# | export


@patch
def _arrange_mirrors(self: Tester) -> None:
    """
    Arranges the mirror functions.

    Iterates over the FastKafka application and its producers and consumers. For each consumer, it retrieves the mirror
    function from the `self.mirrors` dictionary and sets it as an attribute on the Tester instance. It also sets the
    sugar function using the `set_sugar` function. For each producer, it retrieves the mirror function and sets it as
    an attribute on the Tester instance. It also sets the sugar function for the awaited mocks. Finally, it creates the
    `mocks` and `awaited_mocks` namedtuples and sets them as attributes on the Tester instance.

    Returns:
        None
    """
    topic_brokers: Dict[str, Tuple[List[str], List[str]]] = {}
    mocks = {}
    awaited_mocks = {}
    for app in self.apps:
        for topic, (consumer_f, _, _, brokers, _) in app._consumers_store.items():
            mirror_f = self.mirrors[consumer_f]
            self.mirrors[getattr(app, consumer_f.__name__)] = mirror_f
            set_sugar(
                tester=self,
                prefix="to_",
                topic_brokers=topic_brokers,
                topic=remove_suffix(topic),
                brokers=brokers.model_dump_json()
                if brokers is not None
                else app._kafka_brokers.model_dump_json(),
                origin_function_name=consumer_f.__name__,
                function=mirror_f,
            )

            mocks[f"to_{remove_suffix(topic)}"] = getattr(self.mocks, mirror_f.__name__)
            awaited_mocks[f"to_{remove_suffix(topic)}"] = getattr(
                self.awaited_mocks, mirror_f.__name__
            )

        for topic, (producer_f, _, brokers, _) in app._producers_store.items():
            mirror_f = self.mirrors[producer_f]
            self.mirrors[getattr(app, producer_f.__name__)] = getattr(
                self.awaited_mocks, mirror_f.__name__
            )
            set_sugar(
                tester=self,
                prefix="on_",
                topic_brokers=topic_brokers,
                topic=remove_suffix(topic),
                brokers=brokers.model_dump_json()
                if brokers is not None
                else app._kafka_brokers.model_dump_json(),
                origin_function_name=producer_f.__name__,
                function=getattr(self.awaited_mocks, mirror_f.__name__),
            )
            mocks[f"on_{remove_suffix(topic)}"] = getattr(self.mocks, mirror_f.__name__)
            awaited_mocks[f"on_{remove_suffix(topic)}"] = getattr(
                self.awaited_mocks, mirror_f.__name__
            )

    AppMocks = collections.namedtuple(  # type: ignore
        f"{self.__class__.__name__}Mocks", [f_name for f_name in mocks]
    )
    setattr(self, "mocks", AppMocks(**mocks))
    setattr(self, "awaited_mocks", AppMocks(**awaited_mocks))

In [ ]:
# Test mock calls immutability

class Currency(BaseModel):
    currency: str = Field(..., description="Currency")

app = FastKafka(kafka_brokers=dict(localhost=dict(url="server_2", port=9092)))

@app.consumes(prefix="on", topic="store_product")
async def on_store_product(msg: Currency):
    msg.currency = "EUR"
    
async with Tester(app).using_inmemory_broker() as tester:
    await tester.to_store_product(Currency(currency="HRK"))
    await app.awaited_mocks.on_store_product.assert_called_with(
        Currency(currency="HRK"),
        timeout=5
    )
print("ok")

[INFO] fastkafka._testing.in_memory_broker: InMemoryBroker._patch_consumers_and_producers(): Patching consumers and producers!
[INFO] fastkafka._testing.in_memory_broker: InMemoryBroker starting
[INFO] fastkafka._application.app: _create_producer() : created producer using the config: '{'bootstrap_servers': 'server_2:9092'}'
[INFO] fastkafka._testing.in_memory_broker: AIOKafkaProducer patched start() called()
[INFO] fastkafka._components.aiokafka_consumer_loop: aiokafka_consumer_loop() starting...
[INFO] fastkafka._components.aiokafka_consumer_loop: aiokafka_consumer_loop(): Consumer created using the following parameters: {'auto_offset_reset': 'earliest', 'max_poll_records': 100, 'bootstrap_servers': 'server_2:9092'}
[INFO] fastkafka._testing.in_memory_broker: AIOKafkaConsumer patched start() called()
[INFO] fastkafka._components.aiokafka_consumer_loop: aiokafka_consumer_loop(): Consumer started.
[INFO] fastkafka._testing.in_memory_broker: AIOKafkaConsumer patched subscribe() called
[

In [ ]:
# Test batch mirroring


class TestMsg(BaseModel):
    msg: str = Field(...)


second_app = FastKafka(kafka_brokers=dict(localhost=dict(url="localhost", port=9092)))


@second_app.consumes()
async def on_preprocessed_signals(msg: TestMsg, meta: EventMetadata):
    await to_predictions(TestMsg(msg="prediction"))


@second_app.produces()
async def to_predictions(prediction: TestMsg) -> List[TestMsg]:
    print(f"Sending prediction: {prediction}")
    return [prediction]


async with Tester(second_app) as tester:
    await tester.to_preprocessed_signals(TestMsg(msg="signal"))
    await tester.awaited_mocks.on_predictions.assert_called(timeout=5)
    tester.mocks.on_predictions.assert_called()
print("ok")

[INFO] fastkafka._testing.in_memory_broker: InMemoryBroker._patch_consumers_and_producers(): Patching consumers and producers!
[INFO] fastkafka._testing.in_memory_broker: InMemoryBroker starting
[INFO] fastkafka._application.app: _create_producer() : created producer using the config: '{'bootstrap_servers': 'localhost:9092'}'
[INFO] fastkafka._testing.in_memory_broker: AIOKafkaProducer patched start() called()
[INFO] fastkafka._application.app: _create_producer() : created producer using the config: '{'bootstrap_servers': 'localhost:9092'}'
[INFO] fastkafka._testing.in_memory_broker: AIOKafkaProducer patched start() called()
[INFO] fastkafka._components.aiokafka_consumer_loop: aiokafka_consumer_loop() starting...
[INFO] fastkafka._components.aiokafka_consumer_loop: aiokafka_consumer_loop(): Consumer created using the following parameters: {'auto_offset_reset': 'earliest', 'max_poll_records': 100, 'bootstrap_servers': 'localhost:9092'}
[INFO] fastkafka._testing.in_memory_broker: AIOKafk

In [ ]:
kafka_brokers_1 = dict(localhost=dict(url="server_1", port=9092))
kafka_brokers_2 = dict(localhost=dict(url="server_2", port=9092))

app = FastKafka(kafka_brokers=kafka_brokers_1)


@app.consumes(topic="preprocessed_signals")
async def on_preprocessed_signals_1(msg: TestMsg):
    print(f"{msg=}")
    await to_predictions(TestMsg(msg="prediction"))


@app.consumes(topic="preprocessed_signals", brokers=kafka_brokers_2)
async def on_preprocessed_signals_2(msg: TestMsg):
    print(f"{msg=}")
    await to_predictions(TestMsg(msg="prediction"))


async with Tester(app) as tester:
    with pytest.raises(Exception) as exception_produce:
        await tester.to_preprocessed_signals(TestMsg(msg="signal"))
    assert (
        exception_produce.value.args[0]
        == "Ambiguous topic: preprocessed_signals, for functions: ['on_preprocessed_signals_1', 'on_preprocessed_signals_2']\nUse Tester.mirrors[app.function] to resolve ambiguity"
    )
    await tester.mirrors[on_preprocessed_signals_1](TestMsg(msg="signal"))

[INFO] fastkafka._testing.in_memory_broker: InMemoryBroker._patch_consumers_and_producers(): Patching consumers and producers!
[INFO] fastkafka._testing.in_memory_broker: InMemoryBroker starting
[INFO] fastkafka._application.app: _create_producer() : created producer using the config: '{'bootstrap_servers': 'server_1:9092'}'
[INFO] fastkafka._testing.in_memory_broker: AIOKafkaProducer patched start() called()
[INFO] fastkafka._application.app: _create_producer() : created producer using the config: '{'bootstrap_servers': 'server_2:9092'}'
[INFO] fastkafka._testing.in_memory_broker: AIOKafkaProducer patched start() called()
[INFO] fastkafka._components.aiokafka_consumer_loop: aiokafka_consumer_loop() starting...
[INFO] fastkafka._components.aiokafka_consumer_loop: aiokafka_consumer_loop(): Consumer created using the following parameters: {'auto_offset_reset': 'earliest', 'max_poll_records': 100, 'bootstrap_servers': 'server_1:9092'}
[INFO] fastkafka._testing.in_memory_broker: AIOKafkaCo

In [ ]:
kafka_brokers_1 = dict(localhost=[dict(url="server_1", port=9092)])
kafka_brokers_1 = dict(localhost=[dict(url="server_2", port=9092)])

app = FastKafka(kafka_brokers=kafka_brokers_1)


@app.produces(topic="predictions")
async def to_predictions_1(prediction: TestMsg) -> TestMsg:
    print(f"Sending prediction: {prediction}")
    return prediction


@app.produces(topic="predictions", brokers=kafka_brokers_2)
async def to_predictions_2(prediction: TestMsg) -> TestMsg:
    print(f"Sending prediction: {prediction}")
    return prediction


async with Tester(app) as tester:
    with pytest.raises(Exception) as exception_consume:
        await tester.on_predictions.assert_called(timeout=5)
    assert (
        exception_consume.value.args[0]
        == "Ambiguous topic: predictions, for functions: ['to_predictions_1', 'to_predictions_2']\nUse Tester.mirrors[app.function] to resolve ambiguity"
    )
    await tester.mirrors[app.to_predictions_1].assert_not_called(timeout=5)

[INFO] fastkafka._testing.in_memory_broker: InMemoryBroker._patch_consumers_and_producers(): Patching consumers and producers!
[INFO] fastkafka._testing.in_memory_broker: InMemoryBroker starting
[INFO] fastkafka._application.app: _create_producer() : created producer using the config: '{'bootstrap_servers': ['server_2:9092']}'
[INFO] fastkafka._testing.in_memory_broker: AIOKafkaProducer patched start() called()
[INFO] fastkafka._application.app: _create_producer() : created producer using the config: '{'bootstrap_servers': 'server_2:9092'}'
[INFO] fastkafka._testing.in_memory_broker: AIOKafkaProducer patched start() called()
[INFO] fastkafka._components.aiokafka_consumer_loop: aiokafka_consumer_loop() starting...
[INFO] fastkafka._components.aiokafka_consumer_loop: aiokafka_consumer_loop(): Consumer created using the following parameters: {'auto_offset_reset': 'earliest', 'max_poll_records': 100, 'bootstrap_servers': ['server_2:9092']}
[INFO] fastkafka._testing.in_memory_broker: AIOKaf

In [ ]:
# Test KafkaEvent mirroring and consumer batching


class TestMsg(BaseModel):
    msg: str = Field(...)


second_app = FastKafka(kafka_brokers=dict(localhost=dict(url="localhost", port=9092)))


@second_app.consumes()
async def on_preprocessed_signals(msg: List[TestMsg]):
    await to_predictions(TestMsg(msg="prediction"))


@second_app.produces()
async def to_predictions(prediction: TestMsg) -> KafkaEvent[TestMsg]:
    print(f"Sending prediction: {prediction}")
    return KafkaEvent(message=prediction, key=b"123")


async with Tester(second_app) as tester:
    await tester.to_preprocessed_signals(TestMsg(msg="signal"))
    await tester.on_predictions.assert_called(timeout=5)
print("ok")

[INFO] fastkafka._testing.in_memory_broker: InMemoryBroker._patch_consumers_and_producers(): Patching consumers and producers!
[INFO] fastkafka._testing.in_memory_broker: InMemoryBroker starting
[INFO] fastkafka._application.app: _create_producer() : created producer using the config: '{'bootstrap_servers': 'localhost:9092'}'
[INFO] fastkafka._testing.in_memory_broker: AIOKafkaProducer patched start() called()
[INFO] fastkafka._application.app: _create_producer() : created producer using the config: '{'bootstrap_servers': 'localhost:9092'}'
[INFO] fastkafka._testing.in_memory_broker: AIOKafkaProducer patched start() called()
[INFO] fastkafka._components.aiokafka_consumer_loop: aiokafka_consumer_loop() starting...
[INFO] fastkafka._components.aiokafka_consumer_loop: aiokafka_consumer_loop(): Consumer created using the following parameters: {'auto_offset_reset': 'earliest', 'max_poll_records': 100, 'bootstrap_servers': 'localhost:9092'}
[INFO] fastkafka._testing.in_memory_broker: AIOKafk

In [ ]:
# Initiate tester with two apps


class TestMsg(BaseModel):
    msg: str = Field(...)


second_app = FastKafka(kafka_brokers=dict(localhost=dict(url="server_2", port=9092)))


@second_app.consumes()
async def on_preprocessed_signals(msg: TestMsg):
    await to_predictions(TestMsg(msg="prediction"))


@second_app.produces()
async def to_predictions(prediction: TestMsg) -> TestMsg:
    print(f"Sending prediction: {prediction}")
    return prediction


async with Tester([app, second_app]) as tester:
    await tester.to_preprocessed_signals(TestMsg(msg="signal"))
    await tester.mirrors[second_app.to_predictions].assert_called(timeout=5)
print("ok")

[INFO] fastkafka._testing.in_memory_broker: InMemoryBroker._patch_consumers_and_producers(): Patching consumers and producers!
[INFO] fastkafka._testing.in_memory_broker: InMemoryBroker starting
[INFO] fastkafka._application.app: _create_producer() : created producer using the config: '{'bootstrap_servers': ['server_2:9092']}'
[INFO] fastkafka._testing.in_memory_broker: AIOKafkaProducer patched start() called()
[INFO] fastkafka._application.app: _create_producer() : created producer using the config: '{'bootstrap_servers': 'server_2:9092'}'
[INFO] fastkafka._testing.in_memory_broker: AIOKafkaProducer patched start() called()
[INFO] fastkafka._application.app: _create_producer() : created producer using the config: '{'bootstrap_servers': 'server_2:9092'}'
[INFO] fastkafka._testing.in_memory_broker: AIOKafkaProducer patched start() called()
[INFO] fastkafka._application.app: _create_producer() : created producer using the config: '{'bootstrap_servers': 'server_2:9092'}'
[INFO] fastkafka.

In [ ]:
class TestMsg(BaseModel):
    msg: str = Field(...)

kafka_brokers_1 = dict(localhost=dict(url="server", port=9092), production=dict(url="prod_server", port=9092))
kafka_brokers_2 = dict(localhost=dict(url="broker_2_server", port=9092), production=dict(url="broker_2_prod_server", port=9092))
app = FastKafka(kafka_brokers=kafka_brokers_1)

@app.consumes()
async def on_preprocessed_signals(msg: TestMsg):
    await to_predictions(TestMsg(msg="prediction"))


@app.produces()
async def to_predictions(prediction: TestMsg) -> TestMsg:
    print(f"Sending prediction: {prediction}")
    return prediction


@app.produces(topic="predictions", brokers=kafka_brokers_2)
async def to_predictions_2(prediction: TestMsg) -> TestMsg:
    print(f"Sending prediction: {prediction}")
    return prediction

In [ ]:
tester = Tester([app])

with InMemoryBroker() as broker:
    async with tester.using_external_broker() as tester1:
        assert tester1.use_in_memory_broker == False
        bootstraps_topics_groups = list(broker.topic_groups.keys())
        # kafka_brokers1 localhost
        assert bootstraps_topics_groups[0][0] == "server:9092"
        assert bootstraps_topics_groups[1][0] == "server:9092"
        # kafka_brokers2 localhost
        assert bootstraps_topics_groups[2][0] == "broker_2_server:9092"
        assert app._kafka_config["bootstrap_servers_id"] == 'localhost'
        assert tester1._kafka_config["bootstrap_servers_id"] == app._kafka_config["bootstrap_servers_id"]

assert tester.use_in_memory_broker == True
        
with InMemoryBroker() as broker:   
    async with tester.using_external_broker(bootstrap_servers_id="production") as tester2:
        assert tester2.use_in_memory_broker == False
        bootstraps_topics_groups = list(broker.topic_groups.keys())
        # kafka_brokers1 production
        assert bootstraps_topics_groups[0][0] == "prod_server:9092"
        assert bootstraps_topics_groups[1][0] == "prod_server:9092"
        # kafka_brokers2 production
        assert bootstraps_topics_groups[2][0] == "broker_2_prod_server:9092"
        
        assert app._kafka_config["bootstrap_servers_id"] == 'production'
        assert tester2._kafka_config["bootstrap_servers_id"] == app._kafka_config["bootstrap_servers_id"]
        
assert tester.use_in_memory_broker == True


[INFO] fastkafka._testing.in_memory_broker: InMemoryBroker._patch_consumers_and_producers(): Patching consumers and producers!
[INFO] fastkafka._testing.in_memory_broker: InMemoryBroker starting
[INFO] fastkafka._application.app: _create_producer() : created producer using the config: '{'bootstrap_servers': 'server:9092'}'
[INFO] fastkafka._testing.in_memory_broker: AIOKafkaProducer patched start() called()
[INFO] fastkafka._application.app: _create_producer() : created producer using the config: '{'bootstrap_servers': 'broker_2_server:9092'}'
[INFO] fastkafka._testing.in_memory_broker: AIOKafkaProducer patched start() called()
[INFO] fastkafka._application.app: _create_producer() : created producer using the config: '{'bootstrap_servers': 'server:9092'}'
[INFO] fastkafka._testing.in_memory_broker: AIOKafkaProducer patched start() called()
[INFO] fastkafka._components.aiokafka_consumer_loop: aiokafka_consumer_loop() starting...
[INFO] fastkafka._components.aiokafka_consumer_loop: aioka

[INFO] fastkafka._testing.in_memory_broker: AIOKafkaConsumer patched stop() called
[INFO] fastkafka._components.aiokafka_consumer_loop: aiokafka_consumer_loop(): Consumer stopped.
[INFO] fastkafka._components.aiokafka_consumer_loop: aiokafka_consumer_loop() finished.
[INFO] fastkafka._testing.in_memory_broker: AIOKafkaProducer patched stop() called
[INFO] fastkafka._testing.in_memory_broker: AIOKafkaProducer patched stop() called
[INFO] fastkafka._testing.in_memory_broker: InMemoryBroker stopping


In [ ]:
tester = Tester([app])

async with tester.using_inmemory_broker() as tester1:
    assert tester1.use_in_memory_broker == True
    await tester1.to_preprocessed_signals(TestMsg(msg="signal"))
    await tester1.mirrors[app.to_predictions].assert_called(timeout=5)

    assert app._kafka_config["bootstrap_servers_id"] == 'localhost'
    assert tester1._kafka_config["bootstrap_servers_id"] == app._kafka_config["bootstrap_servers_id"]

assert tester.use_in_memory_broker == True

async with tester.using_inmemory_broker(bootstrap_servers_id="production") as tester2:
    assert tester2.use_in_memory_broker == True
    await tester2.to_preprocessed_signals(TestMsg(msg="signal"))
    await tester2.mirrors[app.to_predictions].assert_called(timeout=5)

    assert app._kafka_config["bootstrap_servers_id"] == 'production'
    assert tester2._kafka_config["bootstrap_servers_id"] == app._kafka_config["bootstrap_servers_id"]
    
assert tester.use_in_memory_broker == True

[INFO] fastkafka._testing.in_memory_broker: InMemoryBroker._patch_consumers_and_producers(): Patching consumers and producers!
[INFO] fastkafka._testing.in_memory_broker: InMemoryBroker starting
[INFO] fastkafka._application.app: _create_producer() : created producer using the config: '{'bootstrap_servers': 'server:9092'}'
[INFO] fastkafka._testing.in_memory_broker: AIOKafkaProducer patched start() called()
[INFO] fastkafka._application.app: _create_producer() : created producer using the config: '{'bootstrap_servers': 'broker_2_server:9092'}'
[INFO] fastkafka._testing.in_memory_broker: AIOKafkaProducer patched start() called()
[INFO] fastkafka._application.app: _create_producer() : created producer using the config: '{'bootstrap_servers': 'server:9092'}'
[INFO] fastkafka._testing.in_memory_broker: AIOKafkaProducer patched start() called()
[INFO] fastkafka._components.aiokafka_consumer_loop: aiokafka_consumer_loop() starting...
[INFO] fastkafka._components.aiokafka_consumer_loop: aioka

[INFO] fastkafka._testing.in_memory_broker: AIOKafkaProducer patched stop() called
[INFO] fastkafka._testing.in_memory_broker: AIOKafkaConsumer patched stop() called
[INFO] fastkafka._components.aiokafka_consumer_loop: aiokafka_consumer_loop(): Consumer stopped.
[INFO] fastkafka._components.aiokafka_consumer_loop: aiokafka_consumer_loop() finished.
[INFO] fastkafka._testing.in_memory_broker: AIOKafkaProducer patched stop() called
[INFO] fastkafka._testing.in_memory_broker: AIOKafkaProducer patched stop() called
[INFO] fastkafka._testing.in_memory_broker: InMemoryBroker stopping
